<a href="https://colab.research.google.com/github/fawazshah/News-Media-Reliability/blob/master/train_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:
!pip install transformers

     |████████████████████████████████| 2.2MB 12.6MB/s 
     |████████████████████████████████| 3.3MB 62.8MB/s 
     |████████████████████████████████| 870kB 56.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=44d7a29e3099a1aa1cb4086e7b366b4461294985d9f3bf70f2668873e5acafb4
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [27]:
import json
import numpy as np
import pandas as pd
import random
import requests
from sklearn.metrics import f1_score, accuracy_score, classification_report
import time
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [3]:
# Setting random seed and device
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

### Utils

In [4]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout

### Loading data

In [5]:
corpus_url = 'https://raw.githubusercontent.com/fawazshah/News-Media-Reliability/master/data/emnlp18/corpus-modified.tsv'

corpus = pd.read_csv(corpus_url, sep='\t')
urls = corpus['source_url_normalized'].values

# Ground truths
biases = corpus['bias'].values

In [7]:
article_data_json_url = 'https://raw.githubusercontent.com/fawazshah/News-Media-Reliability/master/data/scraped_articles_1654.json'

r = requests.get(article_data_json_url)
article_data = r.json()

In [49]:
all_data_df = pd.DataFrame(columns=['article headline', 'article body', 'publish date', 'bias'])

news_sources_scraped = 0

for row in corpus.itertuples():
    url = row.source_url_normalized
    bias = row.bias
    if article_data["newspapers"][url] is not None:
        articles = article_data["newspapers"][url].get("articles", [])
        if len(articles) > 0:
            news_sources_scraped += 1
            for article in articles:
                all_data_df = all_data_df.append({'article headline': article['title'],
                                                  'article body': article['text'],
                                                  'publish date': article['published'],
                                                  'bias': bias}, ignore_index=True)

all_data_df['publish date'] = pd.to_datetime(all_data_df['publish date'], utc=True).dt.date

In [50]:
all_data_df

,article headline,article body,publish date,bias
0,On the Ground at the Inauguration: The Only Th...,"Will Sennott\n\nWEDNESDAY, JANUARY 20, 2021, W...",2021-01-22,left
1,"Under President Biden, Will the Yankees Return...",Thurman Munson and Reggie Jackson in 1977 From...,2021-01-21,left
2,Gun Rights Absolutists Celebrate Martin Luther...,"Will Sennott\n\nMONDAY, JANUARY 18, 2021, RICH...",2021-01-19,left
3,Thugs in Blue,"THE BEAT GOES ON … AND ON\n\nOnce Again, Polic...",2021-01-07,left
4,HELL YEAH! Sheriff Clark Publicly DISEMBOWELS ...,Al Sharpton always has had a couple screws loo...,2017-01-13,right
...,...,...,...,...
1649,UK Educators Rank-and-File Safety Committee di...,The UK Educators Rank-and-File Safety Committe...,2021-01-26,left
1650,Make It Sing,Before I lay into the Democrats for missed opp...,2021-01-17,left
1651,Bill Maher: The SPIN Interview,If you care at all about democracy and the way...,2021-01-15,left
1652,Stephan Jenkins on What Culture Truly Means,"“When bad men combine, the good must associate...",2020-06-22,left


### Text preprocessing

In [ ]:
# Text preprocessing preparation

stop_words = ["the", "a", "an", "as", "this", "that", "is", "and", "or", "on",
              "at", "to", "in", "by", "than", "of", "for", "be", "i", "you", 
              "he", "she", "his", "her", "do", "it", "with"]

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

nltk.download('wordnet')

# required for tokenization
nltk.download('punkt')

# required for POS tagging
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# Text preprocessing performed on both article headline and article body

def preprocess(sentence):

    # Lowercase
    sentence = sentence.lower()

    # Punctuation, whitespace removal
    punctuations = '''!()-—[]{};:'"“”‘’\,<>./?@#$%^&*_~'''
    whitespace = '''\n\t'''

    for ch in sentence: 
        if ch in punctuations: 
            sentence = sentence.replace(ch, "")
        if ch in whitespace:
            sentence = sentence.replace(ch, " ")

    # Stop word removal
    remaining_words = []
    
    for word in sentence.split():
        if word not in stop_words:
            remaining_words.append(word)

    sentence = " ".join(remaining_words)

    # Lemmatization
    lemmatized_words = []

    # In order to lemmatise we must first POS-tag each sentence
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)

    for word, tag in tagged:
        pos = nltk_tag_to_wordnet_tag(tag) 
        if pos is not None:
            word = lemmatizer.lemmatize(word, pos=pos)

        lemmatized_words.append(word)

    sentence = " ".join(lemmatized_words)
    
    return sentence

In [ ]:
start = time.time()
all_data_df['article headline'] = all_data_df['article headline'].apply(preprocess)
print(f"Preprocessing headlines took {time.time() - start} seconds")

start = time.time()
all_data_df['article body'] = all_data_df['article body'].apply(preprocess)
print(f"Preprocessing article bodies took {time.time() - start} seconds")

Preprocessing headlines took 1.1521203517913818 seconds
Preprocessing article bodies took 53.263824701309204 seconds


In [ ]:
# Encode labels as numbers
# center == 0
# left == 1
# right == 2

def encode_labels(label):
    if label == "center":
        return 0
    elif label == "left":
        return 1
    else:
        return 2

all_data_df['bias'] = all_data_df['bias'].apply(encode_labels)

In [ ]:
all_data_df

,article headline,article body,bias
0,ground inauguration only thing see be hope,will sennott wednesday january 20 2021 washing...,1
1,under president biden will yankees return thei...,thurman munson reggie jackson 1977 from villag...,1
2,gun right absolutists celebrate martin luther ...,will sennott monday january 18 2021 richmond v...,1
3,thug blue,beat go … once again police pummel plan reform...,1
4,hell yeah sheriff clark publicly disembowel ra...,al sharpton always have have couple screw loos...,2
...,...,...,...
1649,uk educator rankandfile safety committee discu...,uk educator rankandfile safety committee meeti...,1
1650,make sing,before lay into democrat miss opportunity hous...,1
1651,bill maher spin interview,if care all about democracy way our world work...,1
1652,stephan jenkins what culture truly mean,when bad men combine good must associate else ...,1


### Selecting beginning vs middle vs end of body text

In [ ]:
def get_middle_512(sentence):
    toks = sentence.split()
    midpoint = len(toks) // 2
    return ' '.join(toks[midpoint:midpoint+512])

In [ ]:
def get_end_512(sentence):
    toks = sentence.split()
    return ' '.join(toks[-512:])

In [ ]:
# "BEGINNING" is true by default i.e. BERT will just truncate the body to the first 512 tokens
MIDDLE = False
END = False

if MIDDLE:
    all_data_df['article body'] = all_data_df['article body'].apply(get_middle_512)
elif END:
    all_data_df['article body'] = all_data_df['article body'].apply(get_end_512)

In [ ]:
all_data_df

,article headline,article body,bias
0,ground inauguration only thing see be hope,will sennott wednesday january 20 2021 washing...,1
1,under president biden will yankees return thei...,thurman munson reggie jackson 1977 from villag...,1
2,gun right absolutists celebrate martin luther ...,will sennott monday january 18 2021 richmond v...,1
3,thug blue,beat go … once again police pummel plan reform...,1
4,hell yeah sheriff clark publicly disembowel ra...,al sharpton always have have couple screw loos...,2
...,...,...,...
1649,uk educator rankandfile safety committee discu...,uk educator rankandfile safety committee meeti...,1
1650,make sing,before lay into democrat miss opportunity hous...,1
1651,bill maher spin interview,if care all about democracy way our world work...,1
1652,stephan jenkins what culture truly mean,when bad men combine good must associate else ...,1


### Resampling

In [ ]:
RESAMPLE = False

In [ ]:
# Distribution before
print(all_data_df['bias'].value_counts())

2    657
0    567
1    430
Name: bias, dtype: int64


In [ ]:
count_2, count_0, count_1 = all_data_df['bias'].value_counts()

In [ ]:
all_data_0 = all_data_df[all_data_df['bias'] == 0]
all_data_1 = all_data_df[all_data_df['bias'] == 1]
all_data_2 = all_data_df[all_data_df['bias'] == 2]

# Undersample class 0 and 2 to match class 1

all_data_0_under = all_data_0.sample(count_1)
all_data_2_under = all_data_2.sample(count_1)

if RESAMPLE:
    all_data_df = pd.concat([all_data_1, all_data_0_under, all_data_2_under])

In [ ]:
# Distribution after
print(all_data_df['bias'].value_counts())

2    657
0    567
1    430
Name: bias, dtype: int64


In [ ]:
# Shuffle rows in dataset to mix classes up again
all_data_df = all_data_df.sample(frac=1, random_state=1)
all_data_df.reset_index(drop=True)

,article headline,article body,bias
0,bidens america one nation us versus them,be real our history have be constant struggle ...,2
1,how get covid19 vaccine miamidade broward,online earlier anticipated slot week fill up w...,1
2,arm mob storm capitol building during electora...,presidential election its truly stunning heres...,1
3,frontier ebook release january 2021,free download share distribute shape future yo...,0
4,change date vaccine news angry cricket coach,day news live from budgy smuggler booth,0
...,...,...,...
1649,legal liability loom orgs behind rally incite ...,involve but also people who organize who fundr...,1
1650,merck france pasteur institute end development...,least last may several other pharmaceutical co...,0
1651,anthony mackie responsibility message captain ...,marketing we be r have overdo so they can just...,1
1652,union just get rare bit good news from supreme...,agency fee be common solution freerider proble...,1


### Counting frequency of names in dataset

In [ ]:
print(all_data_df['article headline'].str.count("biden").sum())
print(all_data_df['article body'].str.count("biden").sum())

153
1058


In [ ]:
print(all_data_df['article headline'].str.count("trump").sum())
print(all_data_df['article body'].str.count("trump").sum())

211
1798


### Split data into train/val/test

In [ ]:
# Train/val/test split

TRAIN = 0.7
VAL = 0.1
TEST = 0.2

In [ ]:
split_point_1 = int(TRAIN*len(all_data_df))
split_point_2 = int((TRAIN+VAL)*len(all_data_df))

all_data_train_df = all_data_df.iloc[:split_point_1].copy()
all_data_val_df = all_data_df.iloc[split_point_1:split_point_2].copy()
all_data_test_df = all_data_df.iloc[split_point_2:].copy()

In [ ]:
print(f"Size of training set: {len(all_data_train_df)}")
print(f"Size of validation set: {len(all_data_val_df)}")
print(f"Size of test set: {len(all_data_test_df)}")

Size of training set: 1157
Size of validation set: 166
Size of test set: 331


### BERT setup

In [ ]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Compute the length of the longest sentence in particular column out of
# all train, val and test data
def compute_max_length(df, bert_input_func):

  sentences = bert_input_func(df)

  max_len = 0

  for sent in sentences:

      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(sent, add_special_tokens=True)

      # Update the maximum sentence length.
      max_len = max(max_len, len(input_ids))

  return max_len

In [ ]:
def compute_sentences_headline(df):
    return df['article headline'].values

def compute_sentences_body(df):
    return df['article body'].values

def compute_sentences_headline_body(df):
    return [a + " [SEP] " + b
        for a, b in 
        zip(
            df['article headline'].values, 
            df['article body'].values, 
        )]

BERT_INPUTS = [
    {
        'name': 'headlines',
        'func': compute_sentences_headline,
        'max_len': compute_max_length(all_data_df, compute_sentences_headline)
    },
    {
        'name': 'bodies',
        'func': compute_sentences_body,
        'max_len': 512 # Max body length is always longer than 512
    },
    {
        'name': 'headlines + bodies',
        'func': compute_sentences_headline_body,
        'max_len': 512
    },
]

In [ ]:
def create_bert_dataset(df, bert_input_func, max_sequence_len):
    # Returns a TensorDataset of sequences extracted from df

    token_ids = []
    token_type_ids = [] # segment ids 
    attention_masks = []

    sentences = bert_input_func(df)

    for sent in sentences:
        encoding_dict = tokenizer(sent,
                                  add_special_tokens=True,
                                  max_length=max_sequence_len,
                                  padding='max_length',
                                  truncation=True,
                                  return_token_type_ids = True,
                                  return_attention_mask = True,
                                  return_tensors = 'pt'
                                  )
        token_ids.append(encoding_dict['input_ids'])
        token_type_ids.append(encoding_dict['token_type_ids'])
        attention_masks.append(encoding_dict['attention_mask'])
    
    token_ids = torch.cat(token_ids, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(df['bias'].values)
    
    return TensorDataset(token_ids, token_type_ids, attention_masks, labels)

In [ ]:
def train_BERT(train_dataloader, val_dataloader, model, number_epoch):

    train_loss = []
    valid_loss = []

    optimizer = AdamW(model.parameters(),
                    lr = 2e-5, 
                    eps = 1e-8 
                )

    # Create the learning rate scheduler.
    total_steps = len(train_dataloader) * number_epoch
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, 
                                                num_training_steps=total_steps)

    for epoch in range(1, number_epoch+1):

        # TRAINING

        time0 = time.time()

        model.train()

        epoch_train_loss = 0
        no_observations = 0
        epoch_train_predictions = []
        epoch_train_labels = []

        for batch in train_dataloader:

            # Each batch contains token ids, token type ids, attention masks and labels
            b_token_ids = batch[0].to(device)
            b_token_type_ids = batch[1].to(device)
            b_attention_masks = batch[2].to(device)
            b_labels = batch[3].to(device)

            no_observations = no_observations + b_labels.shape[0]
            
            output = model(b_token_ids, 
                    token_type_ids=b_token_type_ids, 
                    attention_mask=b_attention_masks, 
                    labels=b_labels)

            model.zero_grad()

            loss = output.loss
            logits = output.logits

            predictions = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = b_labels.detach().cpu().numpy()
            epoch_train_predictions.extend(predictions)
            epoch_train_labels.extend(labels)

            loss.backward()
            # Clip the norm of the gradients to 1 to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step() 

            # Update the learning rate using the scheduler
            scheduler.step()  

            epoch_train_loss += loss.item()*b_labels.shape[0]

        epoch_train_loss, epoch_train_acc = epoch_train_loss / no_observations, accuracy_score(epoch_train_labels, epoch_train_predictions)

        # VALIDATION

        epoch_valid_loss, epoch_val_predictions, epoch_val_labels = evaluate_BERT(val_dataloader, model)
        epoch_valid_acc = accuracy_score(epoch_val_labels, epoch_val_predictions)

        # FINALLY

        print(f"Epoch took: {time.time() - time0}")

        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_train_loss:.2f} | Train Accuracy: {epoch_train_acc:.2f} | \
        Val. Loss: {epoch_valid_loss:.2f} | Val. Accuracy: {epoch_valid_acc:.2f} |')

        train_loss.append(epoch_train_loss)
        valid_loss.append(epoch_valid_loss)
    
    return train_loss, valid_loss

In [ ]:
def evaluate_BERT(test_dataloader, model):

    model.eval()
    total_loss = 0
    no_observations = 0
    predictions_all = []
    labels_all = []

    with torch.no_grad():
        for batch in test_dataloader:
            b_token_ids = batch[0].to(device)
            b_token_type_ids = batch[1].to(device)
            b_attention_masks = batch[2].to(device)
            b_labels = batch[3].to(device)

            no_observations += b_labels.shape[0]
            output = model(b_token_ids, token_type_ids=b_token_type_ids, 
                                        attention_mask=b_attention_masks)
            logits = output.logits
            loss = loss_fn(logits, b_labels)

            predictions = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = b_labels.detach().cpu().numpy()
            predictions_all.extend(predictions)
            labels_all.extend(labels)

            total_loss += loss.item()*b_labels.shape[0]
    
    return total_loss / no_observations, predictions_all, labels_all

### Create BERT dataloaders

In [ ]:
dataloaders = {}
BATCH_SIZE = 10

In [ ]:
# Note we manually shuffled the dataset earlier, so we can use SequentialSampler to
# sample instead of RandomSampler during training

for bert_input in BERT_INPUTS:

    # Using all data
    dataloaders[bert_input['name']] = {}
    dataloaders[bert_input['name']]['all_data'] = {}
    train_dataset = create_bert_dataset(all_data_train_df, bert_input['func'], bert_input['max_len'])
    dataloaders[bert_input['name']]['all_data']['train'] = DataLoader(train_dataset, sampler=SequentialSampler(train_dataset), batch_size=BATCH_SIZE)
    val_dataset = create_bert_dataset(all_data_val_df, bert_input['func'], bert_input['max_len'])
    dataloaders[bert_input['name']]['all_data']['val'] = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=BATCH_SIZE)
    test_dataset = create_bert_dataset(all_data_test_df, bert_input['func'], bert_input['max_len'])
    dataloaders[bert_input['name']]['all_data']['test'] = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=BATCH_SIZE)

### Run BERT models

In [ ]:
NUM_EPOCHS = 4

In [ ]:
# Evaluate BERT on non-folded data

for bert_input in BERT_INPUTS:

    print(f"---------")
    print(f"{bert_input['name'].upper()}")
    print(f"---------")

    # Set up a new BERT model
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=3,
        output_attentions = False,
        output_hidden_states = False,
    )
    model.cuda()

    # Train model
    train_dataloader = dataloaders[bert_input['name']]['all_data']['train']
    val_dataloader = dataloaders[bert_input['name']]['all_data']['val']
    train_loss, valid_loss = train_BERT(train_dataloader, val_dataloader, model, NUM_EPOCHS)

    # Test model
    test_dataloader = dataloaders[bert_input['name']]['all_data']['test']
    _, predictions, labels = evaluate_BERT(test_dataloader, model)

    print(classification_report(labels, predictions))


---------
HEADLINES
---------


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch took: 15.49701452255249
| Epoch: 01 | Train Loss: 1.06 | Train Accuracy: 0.45 |         Val. Loss: 0.99 | Val. Accuracy: 0.54 |
Epoch took: 15.365787506103516
| Epoch: 02 | Train Loss: 0.92 | Train Accuracy: 0.60 |         Val. Loss: 0.96 | Val. Accuracy: 0.57 |
Epoch took: 15.36445140838623
| Epoch: 03 | Train Loss: 0.73 | Train Accuracy: 0.71 |         Val. Loss: 0.93 | Val. Accuracy: 0.61 |
Epoch took: 15.394659280776978
| Epoch: 04 | Train Loss: 0.58 | Train Accuracy: 0.80 |         Val. Loss: 0.96 | Val. Accuracy: 0.58 |
              precision    recall  f1-score   support

           0       0.65      0.53      0.59       120
           1       0.38      0.40      0.39        83
           2       0.53      0.61      0.57       128

    accuracy                           0.53       331
   macro avg       0.52      0.51      0.51       331
weighted avg       0.54      0.53      0.53       331

---------
BODIES
---------


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch took: 68.55531048774719
| Epoch: 01 | Train Loss: 1.07 | Train Accuracy: 0.45 |         Val. Loss: 0.99 | Val. Accuracy: 0.49 |
Epoch took: 68.51694226264954
| Epoch: 02 | Train Loss: 0.89 | Train Accuracy: 0.60 |         Val. Loss: 0.96 | Val. Accuracy: 0.54 |
Epoch took: 68.54007720947266
| Epoch: 03 | Train Loss: 0.69 | Train Accuracy: 0.72 |         Val. Loss: 0.97 | Val. Accuracy: 0.58 |
Epoch took: 68.49965858459473
| Epoch: 04 | Train Loss: 0.55 | Train Accuracy: 0.81 |         Val. Loss: 0.99 | Val. Accuracy: 0.57 |
              precision    recall  f1-score   support

           0       0.61      0.60      0.60       120
           1       0.34      0.30      0.32        83
           2       0.56      0.60      0.58       128

    accuracy                           0.53       331
   macro avg       0.50      0.50      0.50       331
weighted avg       0.52      0.53      0.52       331

---------
HEADLINES + BODIES
---------


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch took: 68.5475971698761
| Epoch: 01 | Train Loss: 1.06 | Train Accuracy: 0.45 |         Val. Loss: 1.00 | Val. Accuracy: 0.52 |
Epoch took: 68.52441382408142
| Epoch: 02 | Train Loss: 0.92 | Train Accuracy: 0.56 |         Val. Loss: 1.07 | Val. Accuracy: 0.46 |
Epoch took: 68.51956176757812
| Epoch: 03 | Train Loss: 0.74 | Train Accuracy: 0.69 |         Val. Loss: 0.92 | Val. Accuracy: 0.60 |
Epoch took: 68.51496911048889
| Epoch: 04 | Train Loss: 0.58 | Train Accuracy: 0.79 |         Val. Loss: 0.91 | Val. Accuracy: 0.62 |
              precision    recall  f1-score   support

           0       0.65      0.67      0.66       120
           1       0.49      0.33      0.39        83
           2       0.57      0.68      0.62       128

    accuracy                           0.59       331
   macro avg       0.57      0.56      0.56       331
weighted avg       0.58      0.59      0.58       331



### Examining left vs center vs right (not in report yet)

In [ ]:
left_df = all_data_df[all_data_df['bias'] == 1]
center_df = all_data_df[all_data_df['bias'] == 0]
right_df = all_data_df[all_data_df['bias'] == 2]

In [ ]:
# We train using article bodies since this gave highest accuracy in earlier 
# experiments

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

# Train model
train_dataloader = dataloaders['headlines']['all_data']['train']
val_dataloader = dataloaders['headlines']['all_data']['val']
train_loss, valid_loss = train_BERT(train_dataloader, val_dataloader, model, NUM_EPOCHS)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch took: 9.474628448486328
| Epoch: 01 | Train Loss: 1.07 | Train Accuracy: 0.42 |         Val. Loss: 1.02 | Val. Accuracy: 0.49 |
Epoch took: 9.492562055587769
| Epoch: 02 | Train Loss: 0.91 | Train Accuracy: 0.58 |         Val. Loss: 1.01 | Val. Accuracy: 0.54 |
Epoch took: 9.4595468044281
| Epoch: 03 | Train Loss: 0.70 | Train Accuracy: 0.72 |         Val. Loss: 1.08 | Val. Accuracy: 0.58 |
Epoch took: 9.436475992202759
| Epoch: 04 | Train Loss: 0.54 | Train Accuracy: 0.80 |         Val. Loss: 1.07 | Val. Accuracy: 0.60 |


In [ ]:
# Create dataloaders for left_df, center_df, right_df, based on just headline text

left_dataset = create_bert_dataset(left_df, compute_sentences_headline, 512)
left_dataloader = DataLoader(left_dataset, sampler=RandomSampler(left_dataset), batch_size=BATCH_SIZE)

center_dataset = create_bert_dataset(center_df, compute_sentences_headline, 512)
center_dataloader = DataLoader(center_dataset, sampler=RandomSampler(center_dataset), batch_size=BATCH_SIZE)

right_dataset = create_bert_dataset(right_df, compute_sentences_headline, 512)
right_dataloader = DataLoader(right_dataset, sampler=RandomSampler(right_dataset), batch_size=BATCH_SIZE)

In [ ]:
_, predictions, labels = evaluate_BERT(left_dataloader, model)
print(classification_report(labels, predictions))

_, predictions, labels = evaluate_BERT(center_dataloader, model)
print(classification_report(labels, predictions))

_, predictions, labels = evaluate_BERT(right_dataloader, model)
print(classification_report(labels, predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.57      0.73       430
           2       0.00      0.00      0.00         0

    accuracy                           0.57       430
   macro avg       0.33      0.19      0.24       430
weighted avg       1.00      0.57      0.73       430

              precision    recall  f1-score   support

           0       1.00      0.82      0.90       567
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

    accuracy                           0.82       567
   macro avg       0.33      0.27      0.30       567
weighted avg       1.00      0.82      0.90       567

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.87      0.93       657

    accuracy        